In [0]:
from PIL import Image
import numpy as np
import scipy
from scipy import ndimage
import matplotlib.pyplot as plt
import re

In [0]:
import os
from glob import glob

In [24]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
archive=glob("/content/drive/My Drive/ECEN689/Challange7" + '/*.jpg')

In [26]:
np.asarray(archive).shape

(1320,)

In [0]:
import pandas as pd
train = pd.read_csv('/content/drive/My Drive/ECEN689/train.csv')
sample= pd.read_csv('/content/drive/My Drive/ECEN689/sample.csv')

In [28]:
print(train.shape)
print(sample.shape)

(792, 2)
(528, 2)


In [0]:
train['Id']= train['Id'].astype('object')
sample['Id']= sample['Id'].astype('object')

In [0]:
filenames1= list(train['Id'])
filenames2= list(sample['Id'])

In [0]:
filename=[]
train_array=[]
test_array=[]
allarray = []

In [32]:
plt.imread(archive[1]).shape

(300, 400, 3)

In [33]:
for i in archive:
  num = re.findall('\d+', i)
  filename.append(num[2])
  image = np.array(ndimage.imread(i, flatten=False))
  if int(num[2]) in filenames1:
    train_array.append(image)
  else:
    test_array.append(image)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  after removing the cwd from sys.path.


In [34]:
np.asanyarray(train_array).shape

(792, 300, 400, 3)

In [35]:
np.asarray(test_array).shape

(528, 300, 400, 3)

In [0]:
# for i in archive:
#   if(plt.imread(i).shape!= (300,400,3)):
#     print(i)

In [0]:
# basewidth = 300
# hsize = 400
# img = Image.open('/content/drive/My Drive/ECEN689/Challange7/49056593.jpg')

# basewidth = 300
# img = Image.open(‘fullsized_image.jpg')
# wpercent = (basewidth / float(img.size[0]))
# hsize = int((float(img.size[1]) * float(wpercent)))
# img = img.resize((basewidth, hsize), PIL.Image.ANTIALIAS)
# img.save(‘resized_image.jpg')

In [0]:
test_array=np.asarray(test_array)
train_array=np.asarray(train_array)

In [39]:
test_array.shape

(528, 300, 400, 3)

In [0]:
test_x_flatten = test_array.reshape(test_array.shape[0], -1).T
train_x_flatten = train_array.reshape(train_array.shape[0], -1).T

In [41]:
train_x_flatten.shape

(360000, 792)

In [0]:
X_train = train_array / 255
X_test = test_array/ 255

In [0]:
filename1=pd.DataFrame(filename, columns=['Id'], dtype='int')

In [44]:
filename1.shape

(1320, 1)

In [45]:
train.head()

,Id,DGCI
0,41367835,-0.320235
1,56194592,-0.322221
2,75713836,-0.312795
3,1169576,-0.312034
4,88155343,-0.318707


In [0]:
train_new= pd.merge(filename1,train, on='Id', how='left')

In [47]:
train_new.shape

(1320, 2)

In [0]:
test_new= pd.merge(filename1,sample, on='Id', how='left')

In [49]:
test_new.shape

(1320, 2)

In [0]:
test_new.dropna(inplace=True)

In [0]:
test1 = test_new.reset_index(drop=True)

In [52]:
test1.shape

(528, 2)

In [0]:
train_new.dropna(inplace=True)

In [54]:
train_new.shape

(792, 2)

In [55]:
y_train= np.array(train_new['DGCI'])
y_test = np.array(test1['DGCI'])
print(y_train.shape)
print(y_test.shape)

(792,)
(528,)


In [56]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D,MaxPool2D
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from keras import optimizers
from sklearn.preprocessing import StandardScaler
import keras
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, regularizers

Using TensorFlow backend.


In [0]:
def baseline_model():
	# create model
  model = Sequential()
  model.add(Conv2D(32, (5,5), input_shape=(300,400,3), activation='relu'))
  model.add(Conv2D(32, (5,5), activation='relu'))
  model.add(MaxPooling2D(3,3))
  model.add(Conv2D(64, (5,5), activation='relu'))
  model.add(Conv2D(64, (5,5), activation='relu'))
  model.add(MaxPooling2D(3,3))
  model.add(Conv2D(128, (5,5), activation='relu'))
  model.add(MaxPooling2D(3,3))
  model.add(Flatten())
  model.add(Dense(units=512, activation='relu'))
  #model.add(Dropout(0.5))
  model.add(Dense(units=256, activation='relu'))
  model.add(Dense(units=64, activation='relu'))
  model.add(Dense(units=1))
  model.compile(loss='mean_squared_error', optimizer='Adam')
  return model

In [0]:
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model,epochs=20, verbose=1)

In [82]:
estimator.fit(X_train,y_train)

Epoch 1/20
792/792 [==============================] - 16s 20ms/step - loss: 0.1262
Epoch 2/20
792/792 [==============================] - 14s 18ms/step - loss: 0.0388
Epoch 3/20
792/792 [==============================] - 14s 18ms/step - loss: 0.0392
Epoch 4/20
792/792 [==============================] - 14s 18ms/step - loss: 0.0307
Epoch 5/20
792/792 [==============================] - 14s 18ms/step - loss: 0.0642
Epoch 6/20
792/792 [==============================] - 14s 18ms/step - loss: 0.0670
Epoch 7/20
792/792 [==============================] - 14s 18ms/step - loss: 0.0601
Epoch 8/20
792/792 [==============================] - 14s 18ms/step - loss: 0.0569
Epoch 9/20
792/792 [==============================] - 14s 18ms/step - loss: 0.0581
Epoch 10/20
792/792 [==============================] - 14s 18ms/step - loss: 0.0496
Epoch 11/20
792/792 [==============================] - 14s 18ms/step - loss: 0.0391
Epoch 12/20
792/792 [==============================] - 14s 18ms/step - loss: 0.0343
E

In [83]:
z = estimator.predict(X_test)

528/528 [==============================] - 5s 9ms/step


In [84]:
z

array([0.49095532, 0.49464542, 0.50567883, 0.45901632, 0.4895368 ,
       0.52707887, 0.5062403 , 0.49254122, 0.4504764 , 0.5023916 ,
       0.45781893, 0.50131243, 0.47397265, 0.5458783 , 0.5126327 ,
       0.47622278, 0.5238508 , 0.5142882 , 0.50688267, 0.51826715,
       0.45095965, 0.43433133, 0.51833856, 0.5200976 , 0.5179781 ,
       0.52494776, 0.52951944, 0.4740079 , 0.44518155, 0.47933692,
       0.5035112 , 0.4195487 , 0.5350499 , 0.5185431 , 0.52027345,
       0.55641377, 0.45101732, 0.5511223 , 0.4821322 , 0.49497285,
       0.5163219 , 0.49650016, 0.49707115, 0.5129744 , 0.51432085,
       0.500015  , 0.5101867 , 0.48805574, 0.5245855 , 0.47544223,
       0.48997438, 0.3967025 , 0.50477934, 0.5034409 , 0.52593935,
       0.51207864, 0.47417507, 0.53898096, 0.51277995, 0.53397906,
       0.5012194 , 0.46893802, 0.5006707 , 0.39719254, 0.5140543 ,
       0.5239874 , 0.4524228 , 0.47519195, 0.52934873, 0.47807327,
       0.5012924 , 0.50364995, 0.5118201 , 0.49823952, 0.50725

In [85]:
d = {'DGCI':pd.Series(z)}
s = pd.DataFrame(d,index=None)
s.head()

,DGCI
0,0.490955
1,0.494645
2,0.505679
3,0.459016
4,0.489537


In [86]:
test1['DGCI'] = s
test1.head()

,Id,DGCI
0,25715192,0.490955
1,25987554,0.494645
2,26238228,0.505679
3,25988116,0.459016
4,26488575,0.489537


In [0]:
test_final= pd.merge(sample,test1, on='Id', how='left')

In [88]:
test_final.head()

,Id,DGCI_x,DGCI_y
0,79783930,0,0.525091
1,40239934,0,0.521778
2,50410534,0,0.513116
3,47165338,0,0.508352
4,16455157,0,0.542756


In [89]:
test2 = test_final[['Id','DGCI_y']]
test2.columns = ['Id','DGCI']
test2.head()

,Id,DGCI
0,79783930,0.525091
1,40239934,0.521778
2,50410534,0.513116
3,47165338,0.508352
4,16455157,0.542756


In [0]:
test2.to_csv('test3.csv',index=False)
from google.colab import files
files.download('test3.csv')